# Decorating Classes

In [1]:
from fractions import Fraction

In [2]:
f = Fraction(2, 3)

In [3]:
f.denominator

3

In [4]:
f.numerator

2

In [5]:
f.speak()

AttributeError: 'Fraction' object has no attribute 'speak'

In [7]:
Fraction.speak = 100

In [8]:
f.speak

100

In [9]:
Fraction.speak = lambda self, message: 'Fraction says: {0}'.format(message)

In [10]:
f.speak('This is a late parrot')

'Fraction says: This is a late parrot'

In [11]:
f2 = Fraction(10, 5)

In [12]:
f2.speak('This parrot is no more.')

'Fraction says: This parrot is no more.'

In [13]:
Fraction.is_integral = lambda self: self.denominator == 1

In [14]:
f1 = Fraction(2, 3)
f2 = Fraction(64, 8)

In [15]:
f1

Fraction(2, 3)

In [16]:
f2

Fraction(8, 1)

In [17]:
f1.is_integral()

False

In [18]:
f2.is_integral()

True

In [20]:
def dec_speak(cls):
    cls.speak = lambda self, message: '{0} says: {1}'.format(self.__class__.__name__, message)
    return cls

In [21]:
Fraction = dec_speak(Fraction)

In [22]:
f1 = Fraction(2, 3)

In [23]:
f1.speak('hello')

'Fraction says: hello'

In [24]:
class Person:
    pass

In [25]:
Person = dec_speak(Person)

In [26]:
p = Person()

In [27]:
p.speak('this works!')

'Person says: this works!'

In [28]:
from datetime import datetime, timezone

In [52]:
def info(self):
    results = []
    results.append('time: {0}'.format(datetime.now(timezone.utc)))
    results.append('Class: {0}'.format(self.__class__.__name__))
    results.append('id: {0}'.format(hex(id(self))))
    for k, v in vars(self).items():
        results.append('{0}: {1}'.format(k, v))
    return results

def debug_info(cls):
    cls.debug = info
    return cls

In [53]:
@debug_info
class Person:
    def __init__(self, name):
        self.name = name

In [55]:
p = Person('John')

In [57]:
p.debug()

['time: 2021-09-05 20:30:56.310569+00:00',
 'Class: Person',
 'id: 0x7f85503b8df0',
 'name: John']

In [58]:
@debug_info
class Automobile:
    def __init__(self, make, model, year, top_speed):
        self.make = make
        self.model = model
        self.year = year
        self.top_speed = top_speed
        self._speed = 0

    @property
    def speed(self):
        return self._speed

    @speed.setter
    def speed(self, new_speed):
        if new_speed > self.top_speed:
            raise ValueError('Speed cannot exceed top_speed.')
        else:
            self._speed = new_speed

In [59]:
favorite = Automobile('Ford', 'Model T', 1908, 45)

In [60]:
favorite.debug()

['time: 2021-09-05 20:35:14.040788+00:00',
 'Class: Automobile',
 'id: 0x7f85503bba00',
 'make: Ford',
 'model: Model T',
 'year: 1908',
 'top_speed: 45',
 '_speed: 0']

In [62]:
favorite.speed = 40

In [63]:
favorite.debug()

['time: 2021-09-05 20:36:04.202132+00:00',
 'Class: Automobile',
 'id: 0x7f85503bba00',
 'make: Ford',
 'model: Model T',
 'year: 1908',
 'top_speed: 45',
 '_speed: 40']

In [64]:
from math import sqrt

In [75]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __abs__(self):
        return sqrt(self.x ** 2 + self.y ** 2)

    def __repr__(self):
        return 'Point({0}, {1})'.format(self.x, self.y)

In [76]:
p1, p2, p3 = Point(2, 3), Point(2, 3), Point(0, 0)

In [77]:
abs(p1)

3.605551275463989

In [78]:
p1

Point(2, 3)

In [79]:
p1 is p2

False

In [80]:
p2 is p3

False

In [81]:
p1 == p2

False

In [92]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __abs__(self):
        return sqrt(self.x ** 2 + self.y ** 2)

    def __repr__(self):
        return 'Point({0}, {1})'.format(self.x, self.y)

    def __eq__(self, other):
        if isinstance(other, Point):
            return self.x == other.x and self.y == other.y
        else:
            return False

    def __lt__(self, other):
        if isinstance(other, Point):
            return abs(self) < abs(other)
        else:
            return NotImplemented

In [93]:
p1, p2, p3 = Point(2, 3), Point(2, 3), Point(0, 0)

In [94]:
p1 == p2

True

In [95]:
p3 < p1

True

In [96]:
p4 = Point(100, 100)

In [97]:
p4 < p1

False

In [98]:
p4 > p1

True

In [100]:
p1 <= p4

TypeError: '<=' not supported between instances of 'Point' and 'Point'

- a <= b iff a < b or a == b
- a > b iff not(a < b) and a != b
- a >= b iff not(a < b)  

In [101]:
def complete_ordering(cls):
    if '__eq__' in dir(cls) and '__lt__' in dir(cls):
        cls.__le__ = lambda self, other: self < other or self == other
        cls.__gt__ = lambda self, other: not (self < other) and not (self == other)
        cls.__ge__ = lambda self, other: not (self < other)
    return cls

In [102]:
@complete_ordering
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __abs__(self):
        return sqrt(self.x ** 2 + self.y ** 2)

    def __repr__(self):
        return 'Point({0}, {1})'.format(self.x, self.y)

    def __eq__(self, other):
        if isinstance(other, Point):
            return self.x == other.x and self.y == other.y
        else:
            return False

    def __lt__(self, other):
        if isinstance(other, Point):
            return abs(self) < abs(other)
        else:
            return NotImplemented

In [103]:
p1, p2, p3, p4 = Point(2, 3), Point(2, 3), Point(0, 0), Point(100, 200)

In [104]:
p1 <= p4

True

In [105]:
p4 >= p2

True

In [106]:
p1 != p2

False

In [107]:
from functools import total_ordering

In [108]:
@total_ordering
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __abs__(self):
        return sqrt(self.x ** 2 + self.y ** 2)

    def __repr__(self):
        return 'Point({0}, {1})'.format(self.x, self.y)

    def __eq__(self, other):
        if isinstance(other, Point):
            return self.x == other.x and self.y == other.y
        else:
            return False

    def __lt__(self, other):
        if isinstance(other, Point):
            return abs(self) < abs(other)
        else:
            return NotImplemented

In [109]:
p1, p2, p3, p4 = Point(2, 3), Point(2, 3), Point(0, 0), Point(100, 200)

In [110]:
p1 <= p2

True

In [111]:
p1 <= p4

True

In [112]:
p1 >= p4

False

In [113]:
@total_ordering
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __abs__(self):
        return sqrt(self.x ** 2 + self.y ** 2)

    def __repr__(self):
        return 'Point({0}, {1})'.format(self.x, self.y)

    def __eq__(self, other):
        if isinstance(other, Point):
            return self.x == other.x and self.y == other.y
        else:
            return False

    def __gt__(self, other):
        if isinstance(other, Point):
            return abs(self) > abs(other)
        else:
            return NotImplemented

In [114]:
p1, p2, p3, p4 = Point(2, 3), Point(2, 3), Point(0, 0), Point(100, 200)

In [116]:
p4 > p1

True

In [117]:
p2 <= p3

False